In [22]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rc
rc('text',usetex=True)
rc('font',**{'family':'serif','serif':['Computer Modern']})
plt.rc('xtick',labelsize=14)
plt.rc('ytick',labelsize=14)
font={'weight':'bold','size':14}
plt.rc('font',**font)
import time
from scipy import linalg

# Aufgabe 2
# a/b/c
\begin{equation}
y_i=x_{i+1}-x_i-l_i   \tag{1}
\end{equation}

\begin{equation}
m_1\"x_1=k_1(x_2-x_1-l_1)  \rightarrow \qquad m_1\"x_1=k_1y_1  \tag{2}
\end{equation}

\begin{equation}
m_i\"x_i=k_i(x_{i+1}-x_i-l_i)-k_{i-1}(x_i-x_{i-1}-l_{i-1})  \rightarrow \qquad m_i\"x_i=k_iy_i-k_{i-1}y_{i-1}  \tag{3}
\end{equation}


\begin{equation}
m_N\"x_N=-k_{N-1}(x_{N}-x_{N-1}-l_{N-1})   \rightarrow \qquad m_N\"x_N=-k_{N-1}y_{N-1}  \tag{4}
\end{equation}

\begin{equation}
y_i=A_ie^{i\omega t} \qquad   \"y_i=-\omega^2y_i            \tag{5}
\end{equation}




In [130]:
def Construct_Matrix(mi,ki):
    N=len(mi)
    mat_k=np.zeros((N,N))
    mat_k[0,0]=(ki[0]/mi[1]+ki[0]/mi[0])
    mat_k[0,1]=-ki[1]/mi[1]     #first
    for i in range(1,N-1):
        mat_k[i,i-1]=-k[i-1]/m[i]
        mat_k[i,i]=k[i]/m[i]+k[i]/m[i+1]
        mat_k[i,i+1]=-k[i+1]/m[i+1]
    mat_k[N-1,N-2]=k[N-2]/m[N-2]
    mat_k[N-1,N-1]=k[N-2]/m[N-1]
    return mat_k

N=3
m = np.array([i for i in range(1,N+1)])
k=[]
k.append(1.0)
for n in range(1,N-1):
    k.append(1.0/(abs(5-n)+1.0))
k_mat=Construct_Matrix(m,k)
print(k_mat)
eigVal=linalg.eigvals(k_mat)
eig_real=np.real(eigVal)
print(eig_real)
eigenfreq=np.sqrt(eig_real)
print(eigenfreq)

IndexError: list index out of range

In [ ]:
def Max_off_Diag(MatrixK):
    n=len(MatrixK)
    max_off_diag=0.0
    k=0
    l=0
    for i in range(1,n):       # all the rows exept first and last
        for j in range(0,i):
            if abs(MatrixK[i,j])>max_off_diag:
                max_off_diag=abs(MatrixK[i,j])
                k=i
                l=j
    return max_off_diag,k,l

def Jacobi_rotation(MatrixK,N_iter,eps):
    n=len(MatrixK)
    eig_Vec=np.eye(n)
    for i in range(N_iter):
        max_off_diag,k,l=Max_off_Diag(MatrixK)
        if max_off_diag<eps:
            return np.diag(MatrixK),eig_Vec

        phi=0.5*np.arctan2(2.0*MatrixK[k,l],MatrixK[l,l]-MatrixK[k,k])
        c=np.cos(phi)
        s=np.sin(phi)
        P=np.eye(n)
        P[k,k]=c
        P[l,l]=c
        P[k,l]=-s
        P[l,k]=s
        MatrixK=P.T@MatrixK@P
        eig_Vec=eig_Vec@P
    return np.diag(MatrixK),eig_Vec          

For some reason, they are different